In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import face_recognition
import argparse
import sqlite3
import base64
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
models_dir = os.path.join(os.path.dirname(
    os.getcwd()), "project", "base", "models")
prototxtPath = os.path.join(
    models_dir, "Resnet_SSD_deploy.prototxt")
weightsPath = os.path.join(
    models_dir, "Res10_300x300_SSD_iter_140000.caffemodel")
faceNet = cv2.dnn.readNetFromCaffe(
    prototxtPath, weightsPath)

db_dir = os.path.join(os.path.dirname(
    os.getcwd()), "project", "db.sqlite3")

In [3]:
fontFace = cv2.FONT_HERSHEY_SIMPLEX;
fontScale = 0.5;
thickness = 3;

In [4]:
conn = sqlite3.connect(db_dir)
students = pd.read_sql(f"SELECT * FROM school_student WHERE study_class_id == {1}", conn)
users = pd.read_sql(f"SELECT * FROM account_user WHERE id in {tuple(students['user_id'].tolist())}", conn)

In [5]:
students

,student_id,contact,address,face_image,face_encoding,study_class_id,user_id
0,18030073,9867719666,"Butwal, Nepal",face_images/18030073_face.jpg,b'gASVjgQAAAAAAABdlIwVbnVtcHkuY29yZS5tdWx0aWFy...,1,44
1,18030074,9867719666,"Butwal, Nepal",face_images/18030074_face_ly3E2B4.jpg,b'gASVjgQAAAAAAABdlIwVbnVtcHkuY29yZS5tdWx0aWFy...,1,43


In [6]:
users

,id,password,last_login,email,full_name,is_staff,is_active,is_admin
0,43,pbkdf2_sha256$216000$Cud7LVBXGSZQ$u+g2REexrNsQ...,2021-02-18 05:23:53.811151,ajeetkharell@gmail.com,Amit Kharel,0,1,0
1,44,pbkdf2_sha256$216000$oxkMznNSjahq$AiRGeua7NpsY...,2021-02-25 03:24:35.689097,mramitkharel@gmail.com,Ajeet Kharel,0,1,0


In [7]:
def get_encoding(new_encoding):
    np_bytes = base64.b64decode(new_encoding)
    np_array = pickle.loads(np_bytes)
    return np_array[0]

In [8]:
known_face_encodings = students['face_encoding'].apply(get_encoding).to_list()
known_face_ids = students["student_id"].to_list()

In [9]:
def run_stream(cam_idx):
    global encoding
    cap = cv2.VideoCapture(cam_idx)
    while True:
        ret, frame = cap.read()
        if ret:
            (h, w) = frame.shape[:2]
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

            # Convert the image from BGR color (which OpenCV uses) to RGB color (which  face_recognition uses)
            rgb_small_frame = small_frame[:, :, ::-1]

            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # # If a match was found in known_face_encodings, just use the first one.
                # if True in matches:
                #     first_match_index = matches.index(True)
                #     name = known_face_names[first_match_index]

                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_ids[best_match_index]

                face_names.append(name)

            for (top, right, bottom, left), name in zip(face_locations, face_names):
                    # Scale back up face locations since the frame we detected in was scaled to 1/4 size
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4
                
                # Draw a box around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, str(name), (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                    
            cv2.imshow("Face AMS", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [10]:
run_stream(0)

In [11]:
def run_stream(cam_idx):
    global encoding
    cap = cv2.VideoCapture(cam_idx)
    while True:
        ret, frame = cap.read()
        if ret:
            (h, w) = frame.shape[:2]
            frame = cv2.flip(frame, 1)
            
            encoding = face_recognition.face_encodings(frame)

            result = students['face_encoding'].apply(get_similarity)
            student_info = students[result==True].iloc[0]

            (label_width, label_height), baseline = cv2.getTextSize(str(student_info["student_id"]), fontFace, fontScale, thickness)
            cv2.putText(frame, str(student_info["student_id"]), (((startX+endX)//2-label_width), startX), fontFace, fontScale, (255, 0, 0))
                    
            cv2.imshow("Face AMS", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [12]:
run_stream(0)

NameError: name 'get_similarity' is not defined